In [1]:
import csv
import pandas as pd
from tqdm import tqdm
from requests.compat import urljoin
import requests
import glob
import os
import re
from bs4 import BeautifulSoup

In [2]:
# !python -m spacy download en_core_web_sm

In [3]:
# # !pip install spacy
# import spacy

# nlp = spacy.load("en_core_web_sm")

In [4]:
# Open types.txt and read the types
with open('../data/gold_standard_corpus.tsv', 'r') as f:
    pmcids = f.read().splitlines()

In [5]:
# pmcids

In [6]:
import requests
from urllib.parse import urljoin
import csv
from tqdm import tqdm

def get_Json_through_PMCID(pmcid, provider):
    base_url = "https://www.ebi.ac.uk/europepmc/annotations_api/"
    article_url = urljoin(base_url, 
                          f"annotationsByArticleIds?articleIds=PMC%3A{pmcid}&provider={provider}&format=JSON")
    r = requests.get(article_url)

    if r.status_code == 200:
        return r
    else:
        return False

def get_epmc_annotations_to_file(PMCids):
    with open('../data/annotations_from_api.csv', 'w', newline='\n') as f1:
        test_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

        for each_id in tqdm(PMCids):
            # Loop through each provider
            for provider in ["Europe PMC", "PheneBank"]:
                json_annotations = get_Json_through_PMCID(each_id[3:], provider)  # Just the number is needed, so remove the PMC from the front
                if json_annotations:
                    json_results = json_annotations.json()
                    try:
                        pmc_id = json_results[0]['pmcid']
                        for each_annotation in json_results[0]['annotations']:
                            exact = each_annotation['prefix'] + each_annotation['exact'] + each_annotation['postfix']
                            token = each_annotation['tags'][0]['name']
                            ner = each_annotation['type']
                            row = [pmc_id, exact, token, ner, provider]
                            test_writer.writerow(row)
                    except(IndexError):
                        print('No annotations found for ' + str(each_id) + ' from ' + provider)
                else:
                    print('No annotations from ' + provider + ' for ' + str(each_id))
                    continue

# Usage example:
# PMCids = ['PMC123456', 'PMC654321', ...]
# get_epmc_annotations_to_file(PMCids)


In [7]:
# def get_Json_through_PMCID(pmcid):
#     base_url = "https://www.ebi.ac.uk/europepmc/annotations_api/"
#     article_url = urljoin(base_url,
#                           "annotationsByArticleIds?articleIds=PMC%3A" + pmcid + "&provider=Europe%20PMC&format=JSON")
#     r = requests.get(article_url)

#     if r.status_code == 200:
#         return r
#     else:
#         return False


In [8]:
# def get_epmc_annotations_to_file(PMCids):
#     with open('../data/annotations_api.csv', 'w', newline='\n') as f1:
#         test_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

#         # count = 0
#         for each_id in tqdm(PMCids):
#             # count = count+1
#             # print(each_test_pmc_id + '\t' + str(count))
#             json_annotations = get_Json_through_PMCID(each_id[3:])  # Just the number is needed. So remove the PMC from the front
#             if json_annotations:
#                 json_results = json_annotations.json()
#                 try:
#                     pmc_id = json_results[0]['pmcid']
#                     # print(pmc_id)
#                     for each_annotation in json_results[0]['annotations']:
#                         exact = each_annotation['prefix'] + each_annotation['exact'] + each_annotation['postfix']
#                         token = each_annotation['tags'][0]['name']
#                         ner = each_annotation['type']
#                         row = [pmc_id, exact, token, ner]
#                         test_writer.writerow(row)
#                 except(IndexError):
#                     print('no annotations found!! '+str(each_id))
#             else:
#                 print('no annotations! '+str(each_id))
#                 continue


In [9]:
# get_epmc_annotations_to_file(pmcids)

In [10]:
annotations_df = pd.read_csv('../data/annotations_from_api.csv', sep='\t', names=['pmcid', 'partial_sentence', 'token', 'ner', 'provider'])
annotations_df

,pmcid,partial_sentence,token,ner,provider
0,PMC4792959,A subunit of the oligosaccharyltransferase com...,oligosaccharyltransferase complex,Gene Ontology,Europe PMC
1,PMC4792959,"During plant pollen tube (PT) reception, gamet...",pollen tube,Gene Ontology,Europe PMC
2,PMC4792959,"ing in PT rupture, sperm release, and double f...",double fertilization,Gene Ontology,Europe PMC
3,PMC4792959,fic crosses between Arabidopsis thaliana and r...,Arabidopsis thaliana,Organisms,Europe PMC
4,PMC4792959,T3/6 subunit of the oligosaccharyltransferase ...,oligosaccharyltransferase complex,Gene Ontology,Europe PMC
...,...,...,...,...,...
89254,PMC3458065,"plasma viral load, p24-antigen and HIV cap",p24,Gene_Proteins,Europe PMC
89255,PMC3458065,"ad, p24-antigen and HIV capture by erythroc",HIV,Organisms,Europe PMC
89256,PMC3458065,by erythrocytes in HIV-positive individual,HIV,Organisms,Europe PMC
89257,PMC3458065,association between detectable plasma viral lo...,plasma,Anatomy,PheneBank


In [11]:
annotations_df['ner'].unique()

array(['Gene Ontology', 'Organisms', 'Gene_Proteins',
       'Experimental Methods', 'Diseases', 'Chemicals',
       'Accession Numbers', 'Resources', 'Molecule', 'Phenotype',
       'Anatomy', 'Cell', 'Pathway', 'Gene Mutations'], dtype=object)

In [12]:
annotations_df_other = annotations_df[annotations_df['ner'].isin(['Gene Ontology', 'Experimental Methods', 'Accession Numbers', 'Resources', 'Chemicals'])]
annotations_df_other

,pmcid,partial_sentence,token,ner,provider
0,PMC4792959,A subunit of the oligosaccharyltransferase com...,oligosaccharyltransferase complex,Gene Ontology,Europe PMC
1,PMC4792959,"During plant pollen tube (PT) reception, gamet...",pollen tube,Gene Ontology,Europe PMC
2,PMC4792959,"ing in PT rupture, sperm release, and double f...",double fertilization,Gene Ontology,Europe PMC
4,PMC4792959,T3/6 subunit of the oligosaccharyltransferase ...,oligosaccharyltransferase complex,Gene Ontology,Europe PMC
6,PMC4792959,nsferase complex conferring protein N-glycosyl...,glycosylation,Experimental Methods,Europe PMC
...,...,...,...,...,...
89201,PMC3458065,The viral capture assay was performed after t,assay,Experimental Methods,Europe PMC
89220,PMC3458065,ated for 7 to 10 days and assayed for syncytiu...,syncytium formation,Gene Ontology,Europe PMC
89221,PMC3458065,is able to inhibit syncytium formation (loss ...,syncytium formation,Gene Ontology,Europe PMC
89228,PMC3458065,end-point dilution assay in MT-2 cells cultu,assay,Experimental Methods,Europe PMC


In [13]:
# # Get unique PMCIDs
# unique_pmcids = annotations_df_other['pmcid'].unique()url = f"https://www.ebi.ac.uk/europepmc/webservices/rest/{pmcid}/fullTextXML"
# unique_pmcids

# import requests
# from bs4 import BeautifulSoup
# import re

# def get_full_text_xml(pmcid):
#     url = f"https://www.ebi.ac.uk/europepmc/webservices/rest/{pmcid}/fullTextXML"
#     response = requests.get(url)
#     if response.status_code == 200:
#         soup = BeautifulSoup(response.content, 'xml')
#         p_tags = soup.find_all('p')
#         p_texts = [tag.get_text() for tag in p_tags]
#         return p_texts
#     else:
#         return None

In [14]:
unique_pmcids = annotations_df_other['pmcid'].unique()

In [15]:
def get_full_text_xml(pmcid):
    path = f"../data/300_articles_source_files/{pmcid}.xml"
    try:
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        soup = BeautifulSoup(content, 'lxml-xml')  # Use lxml-xml parser
        plain_tags = soup.find_all('plain')
        plain_texts = [tag.get_text() for tag in plain_tags]
        return plain_texts
    except FileNotFoundError:
        print(f"File {pmcid}.xml not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [16]:
import spacy

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

def check_previous_word_pos(sentence, target_word):
    # Process the sentence to get tokenized words and their POS tags
    doc = nlp(sentence)
    
    # Find the target word and get the POS of the previous word
    for i, token in enumerate(doc):
        if token.text == target_word and i > 0:
            previous_word = doc[i - 1]
            pos_of_previous = previous_word.pos_
            
            # Check if the POS is one of the ones we want to avoid
            if pos_of_previous in ['DET', 'ADV', 'VERB', 'ADP']:
                return False  # POS to be avoided
            else:
                return True  # Acceptable POS

    return False  # Return False if the target word is not found or is the first word


In [17]:
def find_sentence_with_substring(string_list, substring):
    for text in string_list:
        sentences = re.split(r'(?<=[.!?])\s+', text)
        for sentence in sentences:
            if substring in sentence:
                return sentence
    return None

def check_previous_word_pos(sentence, prev_word):
    # Process the sentence to get tokenized words and their POS tags
    doc = nlp(sentence)
    
    for token in doc:
        if token.text == prev_word:
            if token.i > 0:
                pos_of_previous = doc[token.i - 1].pos_
                return pos_of_previous not in ['DET', 'ADV', 'VERB', 'ADP', 'NUM', 'CONJ', 'SCONJ']

    return False

def adjust_token_for_context(sentence, token, ner_):
    check_list = ['the', 'an', 'a', 'that', 'after', 'and', 'total', 'using', 'by','as', 'for', 'in', 'with', 
                  'underwent', 'of', 'with', 'but', 'or', 'before', 'at', 
                  'activate', 'research', 'favors', 'prevent', 'to', 'drive', 'on', 'both', 'performed', 'upon',
                 'calculate', 'like', 'μl', 'support', 'our', 'core', 'eight', 'was', 
                  'nested', 'show', 'designed', '3', 'each', 'uts', 'these', 'when' '1', 'no', 
                  'from', 'if', 'individual', 'beyond', 'used', 'facilitated', 
                  'when', 'perform', 'involving', 'time', 'were', 'rich', '1x', 'this', 
                  'Briefly,', 'generated', 'following', 'since']
    
    specific_phrases = ['viral capture assay', 'flow cytometry assay', 
                        'luminescent cell viability assay', 'real-time PCR', 'RT-PCR', 'T-DNA insertion']
    
    
    words = sentence.split(' ')
    punctuation_marks = [',', '.', '?', '!', ';', ']','[',')','(', ':']
    extended_tokens = set()
    
            
    # Remove parentheses from the token
    token_cleaned = token.replace('(', '').replace(')', '')

    # Adjust token for larger term if token is a substring and does not contain specific punctuation
    if ner_ not in['Chemicals', 'Accession Numbers']:
        for word in words:
            cleaned_word = word.replace('(', '').replace(')', '')
            if token_cleaned in cleaned_word and cleaned_word != token_cleaned and not any(mark in cleaned_word for mark in punctuation_marks):
                extended_tokens.add(cleaned_word)
            else:
                extended_tokens.add(token_cleaned)
    else:
        extended_tokens.add(token_cleaned)
            
    

    if ner_ in ['Experimental Methods']:
        
        for phrase in specific_phrases:
        # Find all case-insensitive occurrences of the phrase
            found_phrases = re.findall(phrase, sentence, re.IGNORECASE)
            # Add each found phrase to the extended tokens
            for found in found_phrases:
                extended_tokens.add(found)
    
    # Check for words immediately before the token and add those combinations
        for i, word in enumerate(words):
            cleaned_word = word.replace('(', '').replace(')', '')
            if cleaned_word == token_cleaned and i > 0:
                prev_word = words[i - 1].replace('(', '').replace(')', '')
                if token_cleaned.lower() in ['assay', 'assays', 'insertion', 'insertions', 'elisa', 'pcr', 
                                             'mda', 'iss' 'biopsy', 'co-expression', 
                                             'co-expressions', ' transfection']:
                    if prev_word.lower() not in check_list and (prev_word.isupper() or not prev_word.islower()):
                        extended_tokens.add(prev_word + ' ' + token_cleaned)
                elif prev_word.lower() not in check_list and check_previous_word_pos(sentence, prev_word):
                    extended_tokens.add(prev_word + ' ' + token_cleaned)

        # Always add the token itself, after removing parentheses
        extended_tokens.add(token_cleaned)

    return list(extended_tokens)


def process_pmcid(df, pmcid, p_texts):
    sentences_data = {}
    for _, row in df[df['pmcid'] == pmcid].iterrows():
        sentence = find_sentence_with_substring(p_texts, row['partial_sentence'])
        if sentence:
            if sentence not in sentences_data:
                sentences_data[sentence] = set()

            updated_tokens = adjust_token_for_context(sentence, row['token'], row['ner'])
            for updated_token in updated_tokens:
                sentences_data[sentence].add((updated_token, row['ner']))

    return [[pmcid, sentence, list(ner_tags)] for sentence, ner_tags in sentences_data.items()]

final_data = []

for pmcid in tqdm(unique_pmcids):
    p_texts = get_full_text_xml(pmcid)
    if p_texts:
        processed_data = process_pmcid(annotations_df_other, pmcid, p_texts)
        final_data.extend(processed_data)

# Convert to DataFrame
final_df = pd.DataFrame(final_data, columns=['pmcid', 'sentence', 'ner'])

# Save as TSV
final_df.to_csv('../data/xxx7.tsv', sep='\t', index=False)

100%|█████████████████████████████████████████| 299/299 [00:50<00:00,  5.93it/s]


In [18]:
import ast

def calculate_spans(sentence, entities):
    spans = []
    for entity, etype in entities:
        start = sentence.find(entity)
        if start != -1:
            end = start + len(entity)
            spans.append([start, end, entity, etype])  # List instead of tuple
    return spans

def resolve_overlaps(spans):
    spans.sort(key=lambda x: (x[1] - x[0]), reverse=True)  # Sort by span length
    resolved = []
    for span in spans:
        overlap = False
        for r in resolved:
            if not (span[1] <= r[0] or span[0] >= r[1]):  # Check for overlap
                overlap = True
                break
        if not overlap:
            resolved.append(span)
    return resolved



In [19]:
df = pd.read_csv('../data/xxx7.tsv', sep = '\t')
df['new_ner'] = df.apply(lambda row: resolve_overlaps(calculate_spans(row['sentence'], ast.literal_eval(row['ner']))), axis=1)
df.drop('ner', axis=1, inplace=True)
df.rename(columns={'new_ner': 'ner'}, inplace=True)
df.to_csv('../data/xxx6.tsv', sep = '\t', index=False)

In [20]:
# Rename entity names in file2.csv as per the provided mapping
entity_rename_map = {
    'Gene Ontology': 'GO',
    'Experimental Methods': 'EM', 
    'Accession Numbers': 'AN',
    'Resources': 'RS',
    'Chemicals': 'CD'
}

file2_df = pd.read_csv('../data/xxx6.tsv', sep = '\t')
# Applying the renaming
file2_df['ner'] = file2_df['ner'].apply(lambda x: eval(x))
for row in file2_df['ner']:
    for entity in row:
        entity[3] = entity_rename_map.get(entity[3], entity[3])

# Checking the changes
file2_df.head()


,pmcid,sentence,ner
0,PMC4792959,A subunit of the oligosaccharyltransferase com...,"[[17, 50, oligosaccharyltransferase complex, GO]]"
1,PMC4792959,"During plant pollen tube (PT) reception, gamet...","[[13, 24, pollen tube, GO]]"
2,PMC4792959,ARU encodes the OST3/6 subunit of the oligosac...,"[[38, 71, oligosaccharyltransferase complex, GO]]"
3,PMC4792959,Our results suggest that glycosylation pattern...,"[[25, 38, glycosylation, EM]]"
4,PMC4792959,show that ARTUMES regulates pollen tube recogn...,"[[28, 39, pollen tube, GO]]"


In [21]:
# Correcting the code to work with the 'ners' column, use sets to remove duplicates, 
# and applying specific handling for 'AN' type

# Adjusting the safe_eval_list function to check if the input is a string
def safe_eval_list(str_list):
    if isinstance(str_list, str):
        try:
            return eval(str_list)
        except SyntaxError:
            return []
    return str_list

# Applying the adjusted renaming and extraction
file2_df['ner'] = file2_df['ner'].apply(safe_eval_list)
for row in file2_df['ner']:
    for entity in row:
        entity[3] = entity_rename_map.get(entity[3], entity[3])

        
def replace_substrings(input_str):
    substrings_to_replace = ['*/', '18O ', '% ', 'μm', 'its ', 'are ',  'successful ', 'III ', '6 ', 
                             'their ', 'μm ', 'between ', 'some ', 'through ', 'mL ', 'while ','s layer', '50 ', 
                             'vs ', 'induces ', 'p53 ', 'promotes ', 'within ', 'precise ', 'nitrogen ', '—0', 
                             'other ', 'created ', 'basement ', 'best ', 'study ', 'than ', '94°C,', 'followed ', 
                             '°CAbbreviations:', '7500', 'simultaneous', 'represent', 'II ', '15 ', '10× ', 
                             'initiation ', 'all ', 'toward ', 'improve ', 'whose ', 
                             'influence ', 'convey', 'and/or ', 'acid ', 
                             'analyzed ', 'thus ', 'per ', 'affect ', 'associated ', 'increased' , 
                             'releases ', 'kill', 'Nonetheless, ', 
                            '“', 'against ', 'additional ', 'late ', 'reduced ', 'Distribution ', 
'giving ', 'maintain ', 'nighttime ', 'increase ', 'during ', '30 ' 'HIV ',
'whether ', '- ', 'use ', 'Three ', 'Caused', 'block ', 'IL-8 ', 'No ', 'create ', '”'
'Il-21 ', 'what ', 'base ', 'paid ', 'then ', 'without ', 'where ', '16100 ', '2 ', '325,000 ', 'underlying ', 'related ']
    
    for substring in substrings_to_replace:
        input_str = input_str.replace(substring, '')
    
    return input_str


# Extracting dictionary collection of set with AN, EM, RS, and GO
extracted_entities = {'AN': set(), 'EM': set(), 'RS': set(), 'GO': set(), 'CD': set()}
for row in file2_df['ner']:
    for entity in row:
        entity_type = entity[3]
        if entity_type in extracted_entities:
            # For 'AN', split with a space and take the last term
            if entity_type == 'AN':
                extracted_entities[entity_type].add(entity[2].split()[-1])
            elif entity_type == 'RS' and 'database' in entity[2]:
                extracted_entities[entity_type].add(entity[2].replace('database','').strip())
            elif entity_type == 'GO' or entity_type == 'EM':
                if 'membrane' in entity[2].lower() or 'behaviour' in entity[2].lower():
                    extracted_entities[entity_type].add(entity[2].split()[-1])
                else:
                    replac_str = replace_substrings(entity[2]).strip()
                    if replac_str and len(replac_str)>1:
                        extracted_entities[entity_type].add(replac_str)
            else:
                extracted_entities[entity_type].add(entity[2])


# Converting sets back to lists for easier viewing/usage
extracted_entities = {k: list(v) for k, v in extracted_entities.items()}

# extracted_entities



In [22]:
extracted_entities['RS'].extend(['NHGRI', 'GEO', 'Sequence Read Archive', 'SRA', 'KEGG', 
                                 'Kyoto Encyclopedia of Genes and Genomes', 'IntAct62',
                                 'MINT63', 'AtPIN36', 'AtPID37', 'PAIR38', 'APID39', 'PRIN9', 'PHISTO', 'TAIR',
                                'InParanoid', 'HPRD', 'Medline', 'EMBASE', 'PsycINFO', 'EconLit', 'HEED',
                                 'CRD', 'Health Economic Evaluations Database', 'HEED', 'NHS Economic Evaluation Database',
                                'NHS EED', 'Database of Abstracts of Reviews of Effects', 'DARE', 'PROSITE',
                                'CINAHL', 'Cochrane', 'ARAMEMNON', 'TrEMBL', 'PhosphoSitePlus', 
                                 'UniProtKB/TrEMBL', 'SwissProt', 'miRWalk',
                                 'DAVID', 'SEER', 'Genebank','MSigDB', 'MGI', 'PsyGeNET', 'SFARI', 'DisGeNET',
                                'GeoSymbio', 'TIGR', 'GenBank', 'Swiss-prot' ])

In [23]:
new_accessions = list(set(['FN813615', 'FN813614', 'AM042701', 'AM042698', 'AM042697', 'AM042699', 'FN823053', 
                  'FN995351', 'KY355383', 'ENSDARG000000', 'XP_694951', 'XP_694893', 'XP_686111', 
                  'XP_686816', 'XP_689021', 'XP_693637', 'XP_694145', 'XP_693652', 'XP_691478', 
                  'XP_689023', 'XP_698503', 'XP_690676', 'XP_689954', 'XP_693868', 'XP_688350', 
                  'XP_697815', 'XP_687258', 'XP_690327', 'XP_687591', 'XP_691896', 'XP_689864', 
                  'XP_690860', 'XP_694605', 'XP_694871', 'XP_696432', 'XP_698814', 'XP_699128', 
                  'XP_697396', 'ENSGALG00000011535', 'XM_414969', 'XM_414012', 'XM_414012', 'Q5F3J4', 
                  'ENSXETG00000022012', 'GENSCAN00000015124', 'AC157698', 'ENSXETG00000021586', 
                  'scaffold_279.46c', 'scaffold_1247.5', 'scaffold_3599.1', 'scaffold_626.23', 
                  'scaffold_279.46c', 'scaffold_626.24', 'scaffold_626.25', 'scaffold_279.46c', 
                  'scaffold_279.46c', 'scaffold_111.48', 'ENSXETG00000018490', 'NEWSINFRUG00000144256', 
                  'NEWSINFRUG00000134206', 'NEWSINFRUG00000138086', 'GENSCANSLICE00000000561', 
                  'NEWSINFRUG00000148012', 'TC326097', 'TC353741', 'TC343155', 'EST487', 'MF468321', 
                  'MF468322', 'MF468319', 'MF468318', 'MF468291', 'MF468293', 'MF468299', 'MF468290', 
                  'MF468308', 'MF468291', 'MF468293', 'MF468294', 'MF468298', 'MF468309', 'MF468314', 
                  'MF468295', 'MF468297', 'MF468325', 'MF468328', 'AAL56428', 'AAL56429', 'AAB05820', 
                  'U39815', 'AF126830 ', 'AAL56430', 'AAL58703', 'AAL58704 ', 'rs4179590', 'rs4179590', 
                  'rs4179589', 'rs51960690', 'rs50972129', 'rs4179517', 'rs4179515', 'rs48139623', 
                  'rs32704753', 'rs4179510', 'rs4179504', 'rs4179702', 'rs31418354', 'rs4179917', 
                  'rs4179913', 'rs45694900', 'rs4179908', 'Q1051R', 'rs4179907', 'rs4179728', 
                  'rs4179729', 'rs4179745', 'ENSMUSSNP328538', 'rs32712110', 'rs4179843', 'rs4179872', 
                  'rs31418356', 'rs32712642', 'PRJNA375720', 'AY157059', 'AJ000520', 'BA000042', 
                  'ABE84009', 'ABE86610', 'AE017283', 'NM116255', 'CP000323', 'CP000653', 'AJ316032', 
                  'JQ804931', 'GSE97544', 'PRJEB9815', 'GSM1334015', 'GSM1334016', 'GSM1334017', 
                  'GSM1334018', 'GSM1334019', 'GSM1334020', 'KY355383', 'Q7RTR2', 'Q9Y239', 'Q7RTR2', 
                  'Q86WI3', 'Q7RTR3', 'Q9C000', 'Q9NX02', 'Q96P20', 'Q96MN2', 'P59047', 'P59044', 
                  'Q32MH8', 'Q86W28', 'Q86W27', 'Q86W26', 'P59045', 'P59046', 'Q86W25', 'Q86W24', 
                  'Q9NPP4', 'Q13075', 'P33076', 'KT184354', 'DQ136185', 'AM492684a', 'EU339226', 
                  'EU339209', 'EU339214', 'EU339217', 'Ynys1', 'AF387302a', 'EU339223', 'EU339216', 
                  'WJ68', 'AY455805a', 'EU339224', 'EU339213', 'AY950676a', 'EU339231', 'EU304260a', 
                  'EU339222', 'AY455808a', 'EU339230', 'EU339215', 'GSE24189', 'KJ598498.1', 'GSE9210', 
                  'KF309066', 'MH238461', 'GSE43316', 'AY335912', 'AM902709', 'PRJNA236112', 'AM492684a', 
                  'EU339226', 'EU339209', 'EU339214', 'EU339217', 'Ynys1', 'AF387302a', 'EU339223', 
                  'EU339216', 'WJ68', 'AY455805a', 'EU339224', 'EU339213', 'AY950676a', 'EU339231', 
                  'EU304260a', 'EU339222', 'AY455808a', 'EU339230', 'EU339215', 'EU792508', 'ACF19852', 
                  'EU792509', 'ACF19853', 'EU792510', 'ACF19854', 'EU792511', 'ACF19855', 'AAC32826', 
                  'ABK76312', 'AFJ95132', 'AAC48340', 'AF143817', 'AF041023', 'AB009305', 'AF126830', 
                  'AAL56430', 'AAL58703', 'AAL58704', 'AAL56428', 'AAL56429', 'AAB07368', 'AAB05820', 
                  'U39815', 'AF167707', 'AF167708', 'AF167709', 'AF167710', 'CAC28360', 'U25057', 
                  'BAA14015', 'AAD01628', 'AAB26196', 'P54190', 'U29761', 'AAB53231', 'ACJ64718', 
                  'AAB00227', 'SRP049689', 'EG10787', 'EG11221', 'EG12144', 'EG10379', 'EG10283', 
                  'EG11448', 'EG10688', 'EG10489', 'EG10283', 'EG20173', 'EG10702', 'KX262676.1', 
                  'KX352162', 'KF532967', 'NM_000484', 'KX553930', 'KX553934', 'AM117604', 'FN813615', 
                  'FN813614', 'AM042701', 'AM042698', 'AM042697', 'AM042699', 'FN823053', 'FN995351', 
                  'AM117602.1', 'NM-138554.1', 'P25297', 'PHO84', 'NP_001337.2', 'NP_003638', 'GSE50635','MF468290', 'SRR4011627', 'SRR4011628', 'HM019534', 'HM020374', 'FN813615',
'FN813614', 'AM042701', 'AM042698', 'AM042697', 'AM042699', 'HM019533', 'HM020128', 'HM020130', 'HM020332',
'HM202334', 'HM020374', 'COSM43967', 'COSM1640833', 'COSM562645', 'COSM44853', 'COSM99631']))

In [24]:
len(new_accessions)  #previously 299

258

In [25]:
print(len(extracted_entities['AN']))
extracted_entities['AN'].extend(new_accessions)
print(len(extracted_entities['AN']))

158
416


In [26]:
# with open('../data/CD_terms_list.txt', 'w') as file:
#     for term in list(set(extracted_entities['CD'])):
#         file.write(term + '\n')

In [27]:
chem_to_remove = ['uc.134', 'uc','Yorkie','chemical substances','sec','Cuban','8690176','131127258',
'CD','DS','steward','ZF-EST487-R3','-EST487','delta','n13131313','6429','sc-376167','QX-314','Cy3',
'retina','gamma','singular','electron','1474310','1415180','HM5040',
'singular','sec ','Nile Red ','91558 ','KK4824 ','910111213 ','23333435 ','678545556 ','torques ','dump',
'torque','at a p','syber green','Short','Short chain fatty acids','2021222324','11192732','1001', '90th',
'Gamma',  'Tech', 'fade', ' His-', '-pro-', 'mold', 'electton', 'Psi', 'radio', 'Radio', 'Cho',  'Fmoc', 
 'aaa', 'h groups', 'His-', '3At', 'a steroid', 'Cy3', 'Tris buffers', 'mega', 
 'Halogen', 'Ocn', 'an alcohol', 'fmoc', 'lime', '-Trp', '-Pro', '-His', 'Photon', ' suc', 'nape', 
'Texas red', 'Clo', ' pro-', 'polar amino acids', 'nonyl', 'Dapi', 'hydroxyl radicals', 'aromatic amino acid',
  'Foscarnet', 'mannose', ' Cho', 'at a p', '3 ms', '3 at', 'bile salts', 'procure',  'thio',  'an aldehyde',               
  'Dopa', 'limestones', ' sec', ' Pro-', 'Retina', 'e is', 'flash',   'molds', 'Flash', 'radical', 'torques']

In [28]:
filtered_list = [term for term in list(set(extracted_entities['CD'])) if term not in chem_to_remove]
# filtered_list

In [29]:
# with open('../data/CD_terms_list.txt', 'w') as file:
#     for term in list(set(filtered_list)):
#         file.write(term + '\n')

In [30]:
extracted_entities['GO'].remove('part')
extracted_entities['RS'].remove('HPA')
extracted_entities['CD'] = filtered_list

In [31]:
# import pandas as pd

# def enrich_entities(file_path, new_entity_dict):
#     # Load the CSV file
#     df = pd.read_csv(file_path, names=['sentence', 'ner'])

#     # Function to find if there is an overlap with existing spans or if the term is a substring
#     def is_valid_entity(sentence, new_span, existing_spans, current_entity_type):
#         start_idx, end_idx = new_span
#         # Check for overlap with existing spans and allow 'AN' entities to override 'CD'
#         for span, span_type in existing_spans:
#             if not (end_idx <= span[0] or start_idx >= span[1]):
#                 # Allow 'AN' to override 'CD'
#                 if current_entity_type == 'AN' and span_type == 'CD':
#                     continue
#                 return False
#         # Check if the entity is not part of a larger word (to avoid substrings)
#         if (start_idx > 0 and sentence[start_idx - 1].isalnum()) or (end_idx < len(sentence) and sentence[end_idx].isalnum()):
#             return False
#         return True

#     # Function to add new entity annotations
#     def add_new_entities(sentence, entities, entity_dict):
#         existing_spans = [(tuple(map(int, entity[:2])), entity[3]) for entity in entities if entity != 'None']  # Tuple of span and type
#         for entity_type, terms in entity_dict.items():
#             for term in terms:
#                 start_idx = sentence.find(term)
#                 while start_idx != -1:
#                     end_idx = start_idx + len(term)
#                     new_span = (start_idx, end_idx)
#                     if is_valid_entity(sentence, new_span, existing_spans, entity_type):
#                         entities = [entity for entity in entities if not (entity[0] == start_idx and entity[3] == 'CD')]  # Remove overridden 'CD'
#                         entities.append([start_idx, end_idx, term, entity_type])
#                         existing_spans.append((new_span, entity_type))
#                     start_idx = sentence.find(term, start_idx + 1)
#         return [entity for entity in entities if entity != 'None']  # Exclude 'None' if other entities are present

#     # Enriching the entity annotations
#     for i, row in df.iterrows():
#         sentence = row['sentence']
#         entities = eval(row['ner']) if row['ner'] != "['None']" else ['None']
#         df.at[i, 'ner'] = add_new_entities(sentence, entities, new_entity_dict)

#     return df

import pandas as pd

def enrich_entities(file_path, new_entity_dict, chem_to_remove):
    # Load the CSV file
    df = pd.read_csv(file_path, names=['sentence', 'ner'])

    # Function to find if there is an overlap with existing spans or if the term is a substring
    def is_valid_entity(sentence, new_span, existing_spans, current_entity_type):
        start_idx, end_idx = new_span
        # Check for overlap with existing spans
        for span, span_type in existing_spans:
            if not (end_idx <= span[0] or start_idx >= span[1]):
                return False
        # Check if the entity is not part of a larger word (to avoid substrings)
        if (start_idx > 0 and sentence[start_idx - 1].isalnum()) or (end_idx < len(sentence) and sentence[end_idx].isalnum()):
            return False
        return True

    # Function to add new entity annotations
    def add_new_entities(sentence, entities, entity_dict):
        existing_spans = [(tuple(map(int, entity[:2])), entity[3]) for entity in entities if entity != 'None']  # Tuple of span and type
        for entity_type, terms in entity_dict.items():
            for term in terms:
                start_idx = sentence.find(term)
                while start_idx != -1:
                    end_idx = start_idx + len(term)
                    new_span = (start_idx, end_idx)
                    if is_valid_entity(sentence, new_span, existing_spans, entity_type):
                        entities.append([start_idx, end_idx, term, entity_type])
                        existing_spans.append((new_span, entity_type))
                    start_idx = sentence.find(term, start_idx + 1)

        # Remove 'CD' entities if they are in chem_to_remove list
        entities = [entity for entity in entities if not (entity[2] in chem_to_remove and entity[3] == 'CD')]
        return [entity for entity in entities if entity != 'None']  # Exclude 'None' if other entities are present

    # Enriching the entity annotations
    for i, row in df.iterrows():
        sentence = row['sentence']
        entities = eval(row['ner']) if row['ner'] != "['None']" else ['None']
        df.at[i, 'ner'] = add_new_entities(sentence, entities, new_entity_dict)

    return df


In [32]:
def find_sub_span(sub_span_range, full_spans_range):
    # Check if a sub span is present in full span
    if sub_span_range[0] in range(full_spans_range[0], full_spans_range[1]):
        return sub_span_range

def convert2IOB(text_data, ner_tags):
    if 'None' in ner_tags:
        # Split text into words and return with 'O' tags if there are no NER tags
        split_text = tokenizer.tokenize(text_data)
        return zip(split_text, ['O'] * len(split_text))

    # Tokenize text and create a list of token spans
    tokens = tokenizer.tokenize(text_data)
    current_pos = 0
    token_spans = []
    for token in tokens:
        start_pos = text_data.find(token, current_pos)
        end_pos = start_pos + len(token)
        token_spans.append((start_pos, end_pos))
        current_pos = end_pos

    # Initialize IOB tags as 'O'
    iob_tags = ['O'] * len(tokens)

    # Process each NER tag
    for start, end, entity, entity_type in sorted(ner_tags, key=lambda x: x[0]):
        for i, span in enumerate(token_spans):
            if start <= span[0] < end:
                if start == span[0]:
                    # Beginning of entity
                    iob_tags[i] = 'B-' + entity_type
                else:
                    # Inside of entity
                    iob_tags[i] = 'I-' + entity_type

    return zip(tokens, iob_tags)



In [33]:
import csv
import pathlib
from nltk.tokenize import WordPunctTokenizer, wordpunct_tokenize
tokenizer = WordPunctTokenizer()

def convert_to_IOB_format(data_file, output_folder,filename):
    # Read the CSV file
    df = pd.read_csv(data_file)

    # Define the output path
    result_path = os.path.join(output_folder, filename)
    pathlib.Path(output_folder).mkdir(parents=True, exist_ok=True)
    
    with open(result_path, 'w', newline='\n') as f1:
        train_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')
        
        for index, row in df.iterrows():
            text = row['sentence']
            ner_tags = eval(row['ner']) if row['ner'] != "['None']" else ['None']
            tagged_tokens = convert2IOB(text, ner_tags)
            for each_token in tagged_tokens:
                train_writer.writerow(list(each_token))
            train_writer.writerow('')

output_folder = '../data/enriched_IOB/'

In [42]:
# Example usage
filename = 'train.csv'

file_path = '../data/CD_GP_DS_OG_test/clean_CD_GP_DS_OG_'+filename
new_df = enrich_entities(file_path, extracted_entities, chem_to_remove)
new_df.to_csv(file_path.replace('clean_','enriched_1_'), index=False)

In [43]:
data_file = '../data/CD_GP_DS_OG_test/enriched_1_CD_GP_DS_OG_'+filename
convert_to_IOB_format(data_file, output_folder,filename)